### Which player has the highest number of wins?

In [18]:
import os
from dotenv import load_dotenv, find_dotenv

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

##### MySQL Solution

In [19]:
from tennis_data_analysis.db.mysql import Database
mysql = Database(os.environ.get('DB_HOST'), os.environ.get('DB_USERNAME'), os.environ.get('DB_PASSWORD'), os.environ.get('DB_DATABASE'), os.environ.get('DB_PORT'))

Aggregate sets and find winner per set from GameInfo table

~~~sql
SELECT match_id, set_id, MAX(home_score) AS home_score, MAX(away_score) AS away_score, IF(MAX(away_score) > MAX(home_score), 2, 1) AS winner_code
FROM GameInfo
GROUP BY match_id, set_id
ORDER BY match_id ASC
~~~

**Step 1**
 Making aggregate table for GameInfo table with group by set_id and define winner in every set
~~~sql
SELECT match_id, set_id,
    MAX(home_score) AS home_score,
    MAX(away_score) AS away_score,
    IF(MAX(home_score) > MAX(away_score), 1, 0) AS home_winner,
    IF(MAX(away_score) > MAX(home_score), 1, 0) AS away_winner
FROM GameInfo
GROUP BY match_id, set_id
ORDER BY match_id ASC
~~~

**Step 2**
Insert result into GameInfoSets

In [20]:
sql = """SELECT match_id, set_id,
    MAX(home_score) AS home_score,
    MAX(away_score) AS away_score,
    IF(MAX(home_score) > MAX(away_score), 1, 0) AS home_win,
    IF(MAX(away_score) > MAX(home_score), 1, 0) AS away_win
FROM GameInfo
GROUP BY match_id, set_id
ORDER BY match_id ASC"""
db = mysql.conn().cursor(dictionary=True)
db.execute(sql)
result = db.fetchall()

if result:
    db.execute('TRUNCATE GameInfoSets')
    for row in result:
        sql = 'INSERT INTO GameInfoSets(match_id, set_id, home_score, away_score, home_win, away_win) VALUES(%s,%s,%s,%s,%s,%s)'
        db.execute(sql, (row['match_id'], row['set_id'], row['home_score'], row['away_score'], row['home_win'], row['away_win']))
    mysql.commit()

**Step 3** Find Winner of Every Match from GameInfoSets and insert into GameInfoResult

~~~sql
SELECT s.match_id,
    SUM(s.home_win) AS home_win,
    SUM(s.away_win) AS away_win,
    IFNULL(IF(SUM(s.home_win) > SUM(s.away_win), hp.id, ap.id), 0) AS winner_player_id,
    IFNULL(IF(SUM(s.home_win) < SUM(s.away_win), hp.id, ap.id), 0) AS loser_player_id
FROM GameInfoSets AS s
LEFT JOIN MatchHomeTeamInfo AS h ON h.match_id=s.match_id
LEFT JOIN MatchAwayTeamInfo AS a ON a.match_id=s.match_id
LEFT JOIN Players AS hp ON hp.slug=h.slug
LEFT JOIN Players AS ap ON ap.slug=a.slug
GROUP BY s.match_id
~~~

In [32]:
sql = """SELECT s.match_id,
    SUM(s.home_win) AS home_win,
    SUM(s.away_win) AS away_win,
    IFNULL(IF(SUM(s.home_win) > SUM(s.away_win), hp.id, ap.id), 0) AS winner_player_id,
    IFNULL(IF(SUM(s.home_win) < SUM(s.away_win), hp.id, ap.id), 0) AS loser_player_id
FROM GameInfoSets AS s
LEFT JOIN MatchHomeTeamInfo AS h ON h.match_id=s.match_id
LEFT JOIN MatchAwayTeamInfo AS a ON a.match_id=s.match_id
LEFT JOIN Players AS hp ON hp.slug=h.slug
LEFT JOIN Players AS ap ON ap.slug=a.slug
GROUP BY s.match_id"""
db = mysql.conn().cursor(dictionary=True)
db.execute(sql)
result = db.fetchall()

if result:
    db.execute('TRUNCATE GameInfoResult')
    for row in result:
        sql = 'INSERT INTO GameInfoResult(match_id, home_win, away_win, winner_player_id, loser_player_id) VALUES(%s,%s,%s,%s,%s)'
        db.execute(sql, (row['match_id'], row['home_win'], row['away_win'], row['winner_player_id'], row['loser_player_id']))
    mysql.commit()


**Step 4** Data is prepare in GameInfoResult

In [50]:
sql = """
SELECT p.id AS player_id, p.full_name, p.slug, COUNT(r.winner_player_id) AS num_of_win
FROM GameInfoResult AS r
JOIN Players AS p ON p.id=r.winner_player_id
WHERE r.winner_player_id
GROUP BY r.winner_player_id
ORDER BY num_of_win DESC
LIMIT 1
"""
db.execute(sql)
result = db.fetchone()

if result:
    print("Highest number of win: {}\nPlayer name: {}".format(result['num_of_win'], result['slug']))
else:
    print("Not Found")


Highest number of win: 18
Player name: uchijima-moyuka
